# Tutorial: Leakage scheme implementation

## Author: Leo Werneck

# Initialize NRPy+/Python modules

In [1]:
import os,sys,shutil
import sympy as sp
import astropy.constants as ct
import astropy.units as units
sys.path.append("nrpy_core")
import outputC as outC
import NRPy_param_funcs as par

# Constants & C parameters

We start by implementing a series of constants that will be used by our C code. These appear in the various equations in appendices A and B of [Ruffert *et al.* (1996)](https://adsabs.harvard.edu/pdf/1996A%26A...311..532R). For convenience, we use the [Astropy](https://www.astropy.org/) package to fetch the values of physical constants and for unit conversion.

In [2]:
# Physical constants
h          = ct.h.cgs.to("MeV s").value # MeV s
c          = ct.c.cgs.value             # cm / s
m_e_c2     = ct.m_e.cgs * ct.c.cgs**2   # g cm^2 / s^2
m_e_c2     = m_e_c2.to("MeV").value     # MeV
hc3        = (h*c)**3                   # MeV^3 cm^3
sigma_0    = 1.76e-44                   # cm^{-2}
N_A        = ct.N_A.cgs.value           # 1/mol
gamma_0    = 5.565e-2
alpha      = 1.25                       # Ruffert et al. 1996
Q_npmass   = 1.2935                     # Ruffert et al. 1996
alpha_f    = ct.alpha.value             # Fine-structure constant
Cv         = 0.5 + 2.0*0.23 # Vector coupling
Ca         = 0.5            # Axial coupling
CvCa_aux1  = ((Cv-Ca)**2 + (Cv+Ca)**2)/36.0
CvCa_aux2  = ((Cv-Ca)**2 + (Cv+Ca-2)**2)/9.0
MeV_to_erg = units.MeV.to('erg')
Br_const   = 2.0778e2 / MeV_to_erg
f_Br       = 0.5
amu        = ct.u.cgs.value             # Atomic mass unit (g)

# Neutrino emission and cooling rates





## Fermi-Dirac integrals

Below we implement a function to evaluate Fermi-Dirac integrals

$$
F_{N}(z) = \int_{0}^{\infty}dx\frac{x^{N}}{e^{z-x}+1}\; ,
$$

according to the approximations in [Takahashi, Eid, and Hillebrandt (1978)](https://adsabs.harvard.edu/pdf/1978A%26A....67..185T).

In [75]:
# For N=0, we have an analytic expression
x,z  = sp.symbols('x z',real=True)
one  = sp.sympify(1)
expz = sp.exp(z)
F_0  = sp.simplify(sp.integrate(1.0/(sp.exp(x-z)+1),(x,0,sp.oo)))

# N = 0 case
F_zlarge = [F_0]
F_zsmall = [F_0]

# N=1 case
# z > 1e-3
F_zlarge.append((sp.Rational(1,2)*z**2 + sp.sympify(1.6449))/(one+sp.exp(-sp.sympify(1.6855)*z)))
# z < 1e-3
F_zsmall.append(expz/( one + sp.sympify(0.2159)*sp.exp(sp.sympify(0.8857)*z) ))

# N=2 case
# z > 1e-3
F_zlarge.append((sp.Rational(1,3)*z**3 + sp.sympify(3.2899)*z)/(one - sp.exp(-sp.sympify(1.8246)*z)))
# z < 1e-3
F_zsmall.append(sp.sympify(2)*expz/( one + sp.sympify(0.1092)*sp.exp(sp.sympify(0.8908)*z) ))

# N=3 case
# z > 1e-3
F_zlarge.append((sp.Rational(1,4)*z**4 + sp.sympify(4.9348)*z**2 + sp.sympify(11.3644))/(one+sp.exp(-sp.sympify(1.9039)*z)))
# z < 1e-3
F_zsmall.append(sp.sympify(6)*expz/( one + sp.sympify(0.0559)*sp.exp(sp.sympify(0.9069)*z) ))

# N=4 case
# z > 1e-3
F_zlarge.append((sp.Rational(1,5)*z**5 + sp.sympify(6.5797)*z**3+sp.sympify(45.4576)*z)/(one-sp.exp(-sp.sympify(1.9484)*z)))
# z < 1e-3
F_zsmall.append(sp.sympify(24)*expz/( one + sp.sympify(0.0287)*sp.exp(sp.sympify(0.9257)*z) ))

# N=5 case
# z > 1e-3
F_zlarge.append((sp.Rational(1,6)*z**6 + sp.sympify(8.2247)*z**4 + sp.sympify(113.6439)*z**2 + sp.sympify(236.5323))/(one+sp.exp(-sp.sympify(1.9727)*z)))
# z < 1e-3
F_zsmall.append(sp.sympify(120)*expz/( one + sp.sympify(0.0147)*sp.exp(sp.sympify(0.9431)*z)))

def Cfunc_Fermi_Dirac_Integrals():
    desc = """(c) Leo Werneck
Compute Fermi-Dirac integrals according to the approximations
in Takahashi, Eid, and Hillebrandt (1978)
https://adsabs.harvard.edu/pdf/1978A%26A....67..185T"""
    c_type = "REAL"
    name   = "Fermi_Dirac_integrals"
    params = "const int k, const REAL z"
    outCparams = "outCverbose=False,includebraces=False"
    prefunc = ""
    zlarge_body = ""
    zsmall_body = ""
    for k in range(6):
        zlarge_body += "    case("+str(k)+"):\n"
        zlarge_body += "      "+outC.outputC(F_zlarge[k],"Fermi_Dirac_integral","returnstring",params=outCparams)
        zsmall_body += "    case("+str(k)+"):\n"
        zsmall_body += "      "+outC.outputC(F_zsmall[k],"Fermi_Dirac_integral","returnstring",params=outCparams)
        if k < 5:
            zlarge_body += "      break;\n"
            zsmall_body += "      break;\n"
        else:
            zlarge_body += "      break;"
            zsmall_body += "      break;"
    body   = """
  REAL Fermi_Dirac_integral = 0.0;
  if(z>1e-3) {
    switch(k) {
"""+zlarge_body+r"""
    default:
      fprintf(stderr,"Unsuported value of k: %d\n",k);
      exit(1);
    }
  }
  else {
    switch(k) {
"""+zsmall_body+"""
    default:
      fprintf(stderr,"Unsuported value of k: %d\n",k);
      exit(1);
    }
  }
  
  return Fermi_Dirac_integral;
"""
    loopopts = ""
    outC.outCfunction("Fermi_Dirac_Integrals.cc",
                      desc=desc,prefunc=prefunc,c_type=c_type,name=name,params=params,
                      body=body,loopopts=loopopts,enableCparameters=False)

def Fermi_Dirac_validation(k,z):
    if k > 5:
        print("Unsupported value of k")
        sys.exit(1)  
    if z > 1e-3:
        F = sp.lambdify(sp.symbols('z',real=True),F_zlarge[k])
    else:
        F = sp.lambdify(sp.symbols('z',real=True),F_zsmall[k])
    return F(z)

import fdint
for k in range(6):
    print("Order %d: %e"%(k,(1.0-fdint.fdk(k,1e-5)/Fermi_Dirac_validation(k,1e-5))))
    print("Order %d: %e"%(k,(1.0-fdint.fdk(k,1e-1)/Fermi_Dirac_validation(k,1e-1))))

Order 0: 0.000000e+00
Order 0: 1.110223e-16
Order 1: -3.766628e-05
Order 1: -1.662731e-05
Order 2: 8.862389e-06
Order 2: -5.354045e-05
Order 3: 2.803556e-05
Order 3: 2.572759e-06
Order 4: -1.960806e-05
Order 4: -2.903726e-05
Order 5: -3.869268e-05
Order 5: -3.966739e-07


## Blocking factors

### Electron and positron capture ($\beta$-processes)

For electrons captured by protons, we use (cf. Eqs. B3 and B4 in [Ruffert *et al.* (1996)](https://adsabs.harvard.edu/pdf/1996A%26A...311..532R))
$$
\newcommand{\mue}{\mu_{\rm e}}
\newcommand{\mun}{\mu_{\rm n}}
\newcommand{\mup}{\mu_{\rm p}}
\newcommand{\muhat}{\hat{\mu}}
\newcommand{\me}{m_{\rm e}}
\newcommand{\nb}{n_{\rm b}}
\newcommand{\ee}{e^{-}}
\newcommand{\ae}{e^{+}}
\newcommand{\nue}{\nu_{\rm e}}
\newcommand{\anue}{\bar{\nu}_{\rm e}}
\newcommand{\nui}{\nu_{\rm i}}
\newcommand{\anui}{\bar{\nu}_{\rm i}}
\newcommand{\nux}{\nu_{\rm x}}
\newcommand{\anux}{\bar{\nu}_{\rm x}}
\newcommand{\rhob}{\rho_{\rm b}}
\newcommand{\ye}{Y_{\rm e}}
\newcommand{\QQ}{\mathcal{Q}}
\newcommand{\RR}{\mathcal{R}}
\begin{align}
\langle1-f_{\nue}\rangle_{\rm ec} &= \left\{1 + \exp\left[\eta_{\nue}-\frac{F_{5}(\eta_{\rm e})}{F_{4}(\eta_{\rm e})}\right]\right\}^{-1}\; ,\\
\langle1-f_{\nue}\rangle_{\rm pc} &= \left\{1 + \exp\left[\eta_{\anue}-\frac{F_{5}(-\eta_{\rm e})}{F_{4}(-\eta_{\rm e})}\right]\right\}^{-1}\; ,
\end{align}
$$

where $\eta_{\nui} = \mu_{\nui}/T$, $T$ is the temperatuer, and $\mu_{\nui}$ is the chemical potential of neutrino species $\nui=\bigl\{\nue,\anue,\nux,\anux\bigr\}$, where $x$ represents either the tau or the muon. We set

$$
\eta_{\nux} = 0 = \eta_{\anux}\; ,
$$

and

$$
\begin{align}
\eta_{\nue} &= \eta_{\nue}^{\rm ceq} \left[1 - \exp(-\tau_{\nue})\right] + \eta_{\nue}^{0}\exp(-\tau_{\nue})\; ,\\
\eta_{\anue} &= \eta_{\anue}^{\rm ceq} \left[1 - \exp(-\tau_{\anue})\right] + \eta_{\anue}^{0}\exp(-\tau_{\anue})\; ,
\end{align}
$$

where $\eta_{\nue}^{0} = 0 = \eta_{\anue}^{0}$, $\tau_{\nue}$ and $\tau_{\anue}$ are the optical depths of the electron neutrino and antineutrino, respectively, and

$$
\eta_{\nue}^{\rm ceq} =  - \eta_{\anue}^{\rm ceq} = \eta_{\rm e} + \eta_{\rm p} - \eta_{\rm n} - Q/T\; ,
$$

where $\eta_{i}=\mu_{i}/T$, with $i={\rm e,n,p}$, and $Q=1.2935$ MeV is the rest-mass-energy difference between a neutron and a proton.

In [77]:
# Compute degeneracy parameters
zero = sp.sympify(0)
Q = sp.sympify(1.2935)
mu_e, mu_n, mu_p, mu_hat, T, tau_nue, tau_anue = sp.symbols("mu_e mu_n mu_p mu_hat T tau_nue tau_anue",real=True)
eta_e        = mu_e / T
eta_n        = mu_n / T
eta_p        = mu_p / T
eta_hat      = mu_hat / T
eta_nue_ceq  = eta_e + eta_p - eta_n - Q/T
eta_anue_ceq = -eta_nue_ceq
eta_nue_0    = zero
eta_anue_0   = zero
eta_nux      = zero

# Compute eta_nue and eta_anue
eta_nue  = eta_nue_ceq  * (one - sp.exp(-tau_nue) ) + eta_nue_0 *sp.exp(-tau_nue)
eta_anue = eta_anue_ceq * (one - sp.exp(-tau_anue)) + eta_anue_0*sp.exp(-tau_anue)

# Fermi-Dirac integrals have been implemented above.
# Here we simply call a NRPy+-compatible version.
F_4_e_plus  = outC.nrpyFermiDiracintegrals(4,+eta_e)
F_5_e_plus  = outC.nrpyFermiDiracintegrals(5,+eta_e)
F_4_e_minus = outC.nrpyFermiDiracintegrals(4,-eta_e)
F_5_e_minus = outC.nrpyFermiDiracintegrals(5,-eta_e)

# Now compute the blocking factors
blocking_factor_beta_nue  = one/( one + sp.exp(eta_nue  - F_5_e_plus/F_4_e_plus) )
blocking_factor_beta_anue = one/( one + sp.exp(eta_anue - F_5_e_minus/F_4_e_minus) )

### Electron-positron pair annihilation

We now consider the blocking factor for electorn-positron pair annihilation, i.e.,

$$
\ae + \ee \to \nui + \anui\; ,
$$

which is given by Eq. (B9) in [Ruffert *et al.* (1996)](https://adsabs.harvard.edu/pdf/1996A%26A...311..532R),

$$
\langle1-f_{\nui}\rangle_{\ae,\ee} = \left\{1 + \exp\left[\eta_{\nui} - \frac{1}{2}\frac{F_{4}(\eta_{e})}{F_{3}(\eta_{e})} - \frac{1}{2}\frac{F_{4}(-\eta_{e})}{F_{3}(-\eta_{e})} \right]\right\}^{-1}\; .
$$

In [79]:
# Compute useful quantities
F_4_eta_e_plus  = outC.nrpyFermiDiracintegrals(4,+eta_e)
F_3_eta_e_plus  = outC.nrpyFermiDiracintegrals(3,+eta_e)
F_4_eta_e_minus = outC.nrpyFermiDiracintegrals(4,-eta_e)
F_3_eta_e_minus = outC.nrpyFermiDiracintegrals(3,-eta_e)
common_factor   = sp.Rational(1,2)*( F_4_eta_e_plus/F_3_eta_e_plus + F_4_eta_e_minus/F_3_eta_e_minus )

# Now compute the blocking factors
blocking_factor_pair_nue  = one/( one + sp.exp( eta_nue  - common_factor ))
blocking_factor_pair_anue = one/( one + sp.exp( eta_anue - common_factor ))
blocking_factor_pair_nux  = one/( one + sp.exp( eta_nux  - common_factor ))

### Plasmon decay

Next we consider the blocking factor for plasmon decay,

$$
\gamma \to \nui + \anui\; ,
$$

where $\gamma$ represents the plasmon. We implement Eq. (B13) in [Ruffert *et al.* (1996)](https://adsabs.harvard.edu/pdf/1996A%26A...311..532R),

$$
\langle1-f_{\nui}\rangle_{\gamma} = \left\{1 + \exp\left[\eta_{\nui} - \left(1 + \frac{1}{2}\frac{\gamma^{2}}{1+\gamma}\right)\right]\right\}^{-1}\; ,
$$

where

$$
\gamma = \gamma_{0}\sqrt{\frac{1}{3}\left(\pi^{2}+3\eta_{e}^{2}\right)}\; ,
$$

where $\gamma_{0} = 5.565\times10^{-2}$.

In [83]:
# Define gamma_0
gamma_0 = sp.sympify(5.565e-2)
M_PI    = sp.symbols("M_PI",real=True)

# Compute gamma
gamma = gamma_0 * sp.sqrt(sp.Rational(1,3)*(M_PI**2 + sp.sympify(3)*eta_e**2))

# Compute the blocking factors
common_factor = one + sp.Rational(1,2)*gamma**2/(one+gamma)
blocking_factor_plasmon_nue  = one/( one + sp.exp( eta_nue  - common_factor))
blocking_factor_plasmon_anue = one/( one + sp.exp( eta_anue - common_factor))
blocking_factor_plasmon_nux  = one/( one + sp.exp( eta_nux  - common_factor))

## Energy moments

Next we implement the electron and positron energy moments given by Eqs. (B5), (B6), and (B7) in [Ruffert *et al.* (1996)](https://adsabs.harvard.edu/pdf/1996A%26A...311..532R),

$$
\begin{align}
\varepsilon_{e^{\mp}} \equiv \frac{8\pi}{(hc)^{3}}T^{4}F_{3}(\pm\eta_{e})
\end{align}
$$